In [11]:
import pandas as pd 

file = pd.read_csv('Data/gldas-current_precipitation.csv')

In [12]:
file.head()

,Unnamed: 0,468T8,49HYF,72677,72768,72769,72770,72772,72773,72774,...,KS850,KSDY0,KWYS0,MDSMM,MRYDA,OBMJ4,U1K6U,V83LW,WLH1H,XEEN8
0,2023-01-01,0.000000,0.000030,0.000094,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.00021,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000
1,2023-01-02,0.000000,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000
2,2023-01-03,0.000000,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000
3,2023-01-04,0.000000,0.000000,0.000000,0.0,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000
4,2023-01-05,0.000085,0.000002,0.000053,0.0,0.00024,0.00028,0.000165,0.000652,0.000284,...,0.0,0.0,0.00000,0.0,0.0,0.000013,0.000144,0.00028,0.000144,0.000003


In [13]:
file["468T8"].max()

0.0011845999902106

In [14]:
file.describe()

,468T8,49HYF,72677,72768,72769,72770,72772,72773,72774,72775,...,KS850,KSDY0,KWYS0,MDSMM,MRYDA,OBMJ4,U1K6U,V83LW,WLH1H,XEEN8
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,...,731.000000,731.000000,7.310000e+02,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,0.000105,0.000117,0.000118,0.000088,0.000112,0.000114,0.000118,0.000125,0.000114,0.000109,...,0.000091,0.000096,1.292236e-04,0.000105,0.000103,0.000115,0.000109,0.000114,0.000110,0.000114
std,0.000183,0.000197,0.000208,0.000176,0.000181,0.000188,0.000192,0.000193,0.000180,0.000181,...,0.000186,0.000209,2.170512e-04,0.000199,0.000192,0.000203,0.000181,0.000186,0.000182,0.000209
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000020,0.000019,0.000017,0.000003,0.000019,0.000011,0.000022,0.000016,0.000017,0.000020,...,0.000000,0.000000,4.000001e-07,0.000014,0.000010,0.000014,0.000020,0.000012,0.000020,0.000013
75%,0.000133,0.000162,0.000152,0.000093,0.000165,0.000169,0.000167,0.000207,0.000166,0.000137,...,0.000091,0.000086,1.955628e-04,0.000116,0.000121,0.000146,0.000136,0.000169,0.000134,0.000143
max,0.001185,0.001236,0.001407,0.001605,0.001383,0.001611,0.001174,0.001116,0.001318,0.001192,...,0.001578,0.001706,2.463549e-03,0.001359,0.001264,0.001335,0.001192,0.001476,0.001192,0.001428
